# **TP3**

## **Clustering**

### Imports

In [8]:
# Points 2-3 : Clustering
import collections
import os
import string
import sys

import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine

# Wordcloud
import yake
from collections import Counter
from wordcloud import WordCloud
from IPython.display import Image

# Points 5-7 : Word2Vec
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

In [9]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords') # wordcloud

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anton\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\anton\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anton\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
data_path = "../data/txt/"

### Choisir une décennie

1. Choisissez une décennie entre 1890-1899 et 1960-1969 ➡️ 1930

In [11]:
DECADE = '1930'

In [12]:
# Charger tous les  fichiers de la décennie et en créer une liste de textes
files = [f for f in sorted(os.listdir(data_path)) if f"_{DECADE[:-1]}" in f]

In [13]:
# Exemple de fichiers
files[:5]

['KB_JB838_1930-01-05_01-00006.txt',
 'KB_JB838_1930-01-09_01-00001.txt',
 'KB_JB838_1930-01-09_01-00014.txt',
 'KB_JB838_1930-01-16_01-00010.txt',
 'KB_JB838_1930-01-20_01-00002.txt']

In [ ]:
texts = [open(data_path + f, "r", encoding="utf-8").read() for f in files]

In [ ]:
# Exemple de textes
texts[0][:400]

### Vectoriser les documents à l'aide de TF-IDF

In [ ]:
# Création d'une fonction de pré-traitement
def preprocessing(text, stem=True):
    """ Tokenize text and remove punctuation """
    text = text.translate(string.punctuation)
    tokens = word_tokenize(text)
    return tokens

In [ ]:
# Instancier le modèle TF-IDF avec ses arguments
vectorizer = TfidfVectorizer(
    tokenizer=preprocessing,
    stop_words=stopwords.words('french'),
    max_df=0.5,
    min_df=0.1,
    lowercase=True)

In [ ]:
# Construire la matrice de vecteurs à l'aide de la fonction `fit_transform`
tfidf_vectors = vectorizer.fit_transform(texts)

In [ ]:
# Détail de la matrice
tfidf_vectors

In [ ]:
# Imprimer le vecteur tf-IDF du premier document
pd.Series(
    tfidf_vectors[0].toarray()[0],
    index=vectorizer.get_feature_names_out()
    ).sort_values(ascending=False)

### Comprendre les vecteurs et leurs "distances"

In [ ]:
cosine([1, 2, 3], [1, 2, 3])

In [ ]:
cosine([1, 2, 3], [1, 2, 2])

In [ ]:
cosine([1, 2, 3], [2, 2, 2])

### Tests sur nos documents

In [ ]:
tfidf_array = tfidf_vectors.toarray()

In [ ]:
# Vecteur du document 0
tfidf_array[0]

In [ ]:
# Vecteur du document 1
tfidf_array[1]

In [ ]:
cosine(tfidf_array[0], tfidf_array[1])

### Appliquer un algorithme de clustering sur les vecteurs TF-IDF des documents

2. Effectuez un clustering des documents de cette décennie, en adaptant éventuellement le nombre de clusters désirés.

In [ ]:
# Définir un nombre de clusters
N_CLUSTERS = 3

In [ ]:
# Instancier le modèle K-Means et ses arguments
km_model = KMeans(n_clusters=N_CLUSTERS)

In [ ]:
# Appliquer le clustering à l'aide de la fonction `fit_predict`
clusters = km_model.fit_predict(tfidf_vectors)

In [ ]:
clustering = collections.defaultdict(list)

for idx, label in enumerate(clusters):
    clustering[label].append(files[idx])

In [ ]:
pprint(dict(clustering))

### Visualiser les clusters

In [ ]:
# Réduire les vecteurs à 2 dimensions à l'aide de l'algorithme PCA
pca = PCA(n_components=2)
reduced_vectors = pca.fit_transform(tfidf_vectors.toarray())

In [ ]:
reduced_vectors[:10]

In [ ]:
# Générer le plot
x_axis = reduced_vectors[:, 0]
y_axis = reduced_vectors[:, 1]

plt.figure(figsize=(10,10))
scatter = plt.scatter(x_axis, y_axis, s=100, c=clusters)

# Ajouter les centroïdes
centroids = pca.transform(km_model.cluster_centers_)
plt.scatter(centroids[:, 0], centroids[:, 1],  marker = "x", s=100, linewidths = 2, color='black')

# Ajouter la légende
plt.legend(handles=scatter.legend_elements()[0], labels=set(clusters), title="Clusters")

3. Tentez d'interpréter les résultats obtenus : les clusters semblent-ils faire sens? Si oui/non, comment l'expliquez-vous? Aidez-vous au besoin d'une analyse de keywords et/ou wordcloud (voir TP2).

#### Wordcloud

In [ ]:
# Stopwords (Idem que dans s1)
sw = stopwords.words("french")
sw += ["les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout",
       "ils", "bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous",
       "celle", "entre", "encore", "toutes", "pendant", "moins", "dire", "cela", "non",
       "faut", "trois", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres",
       "van", "het", "autre", "jusqu", "ville", "rossel", "dem"]
sw = set(sw)

In [ ]:
# Identifier le chemin de vos données et spécifier le cluster à traiter
data_path = "../data/txt/"
cluster_id = 0  # changer ce numéro pour le cluster souhaité
files_in_cluster = clustering[cluster_id]  # liste des fichiers dans le cluster

# Concaténer le texte de tous les fichiers du cluster
cluster_text = " ".join([open(os.path.join(data_path, f), "r", encoding="utf-8").read() for f in files_in_cluster])

In [ ]:
# Nettoyer le texte
words = nltk.wordpunct_tokenize(cluster_text)
kept_words = [w.upper() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
cleaned_text = " ".join(kept_words)

# Compter les mots dans le cluster
word_counts = Counter(cleaned_text.split())

In [ ]:
# Générer le wordcloud pour le cluster
wordcloud = WordCloud(width=2000, height=1000, background_color='white').generate_from_frequencies(word_counts)

# Afficher le wordcloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title(f"Wordcloud - cluster n.{cluster_id}")
plt.show()

## **Word2Vec**

4. Téléchargez sur l’UV le fichier zippé sents.txt, déjà segmenté en phrases, et placez-le dans le dossier data ⬇️✔️

### Chargement et traitement des phrases du corpus

In [ ]:
# Création d'un objet qui streame les lignes d'un fichier pour économiser de la RAM
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [ ]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

In [ ]:
bigram_phrases = Phrases(sentences)

In [ ]:
type(bigram_phrases.vocab)

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [ ]:
# Clés / termes observés dans le corpus
len(bigram_phrases.vocab.keys())

In [ ]:
# Prendre une clé au hasard
key_ = list(bigram_phrases.vocab.keys())[2209]
print(key_)

In [ ]:
# Indiquer le score de cette cooccurrence
bigram_phrases.vocab[key_]

In [ ]:
# Conversion des `Phrases` en objet `Phraser`
bigram_phraser = Phraser(phrases_model=bigram_phrases)

### Extraction des trigrams

In [ ]:

trigram_phrases = Phrases(bigram_phraser[sentences])

In [ ]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [ ]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [ ]:
print(corpus[:100])

### Entrainement d'un modèle Word2Vec sur ce corpus
5. Entraînez un modèle word2vec (word embeddings) sur ces phrases grâce au notebook s3_word_embeddings.ipynb, en adaptant éventuellement les paramètres window (taille de la fenêtre) et min_count (nombre minimum d’occurrences d’un mot)

In [ ]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=7, # La taille du "contexte", ici 7 mots avant et après le mot observé
    min_count=3, # On ignore les mots qui n'apparaissent pas au moins 3 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

In [ ]:
#  Sauver le modèle dans un fichier
outfile = f"../data/balanced.model"
model.save(outfile)

6. Vous pouvez entraîner plusieurs modèles afin de comparer leurs performances, en procédant par essais-erreurs pour choisir le meilleur modèle

J'ai entraîné 4 modèles avec les paramètres suivants : 
- "default" : window=5, min_count=5 ;
- "global_r" : window=15, min_count=10 ;
- "local_r" : window=3, min_count=1 ;
- "balanced" : window=7, min_count=3 ;

### Explorer le modèle
7. Explorez le modèle retenu à l’aide des fonctions similarity et most_similar (choisissez au moins trois exemples pour chaque fonction)

In [ ]:
# Charger le modèle en mémoire
model = Word2Vec.load("../data/balanced.model")

In [ ]:
# Termes présents dans les 3 exemples pris en considération
termes_ex = ["vente", "achat", "agence", "maison", "championnat", "orchestre"] # liste avec les termes issus des 3 exemples

# Imprimer le vecteur de chaque terme présent dans les 3 exemples sélectionnés
for terme in termes_ex:
    print(f"{terme} :", model.wv[terme])
    print()

In [ ]:
# Calculer la similarité entre les termes
similarité_ex1 = model.wv.similarity("vente", "achat") # Exemple 1 : relation évidente
similarité_ex2 = model.wv.similarity("agence", "maison") # Exemple 2 : relation indirecte
similarité_ex3 = model.wv.similarity("championnat", "orchestre") # Exemple 3 : relation inexistante

# Affichage des résultats
print("vente - achat :", similarité_ex1)
print()
print("agence - maison :", similarité_ex2)
print()
print("championnat - orchestre :", similarité_ex3)

In [ ]:
# Parcourir la liste des termes et chercher les 10 mots les plus proches pour chacun d'entre eux
for terme in termes_ex:
    print(f"Les mots les plus proches de '{terme}' sont :")
    similar_terms = model.wv.most_similar(terme, topn=10)
    for similar_term, similarity in similar_terms:
        print(f"- '{similar_term}', {similarity:.4f}")
    print()

In [ ]:
# Faire des recherches complexes à travers l'espace vectoriel

recherche_1 = model.wv.most_similar(positive=["vente", "achat"], negative=["agence"], topn=5)
print("Exemple 1 : Trouver un mot qui relie la relation entre 'vente' et 'achat', en excluant la notion d''agence'.")
for word, similarity in recherche_1:
    print(f"- {word}, {similarity:.4f}")
print()

recherche_2 = model.wv.most_similar(positive=["maison", "orchestre"], topn=5)
print("Exemple 2 : Trouver un mot qui puisse associer le concept de 'maison' à celui d''orchestre'")
for word, similarity in recherche_2:
    print(f"- {word}, {similarity:.4f}")
print()

recherche_3 = model.wv.most_similar(positive=["championnat"], negative=["orchestre"], topn=5)
print("Exemple 3 : Trouver des idées liéés au concept de 'championnat' (sport), en excluant la notion d''orchestre'.")
for word, similarity in recherche_3:
    print(f"- {word}, {similarity:.4f}")
print()